<a href="https://colab.research.google.com/github/KazGet/Laboratory-work/blob/main/Text_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '')  # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text)  # заменяем все символы кроме кириллицы на пустые символы

# парсим текст, как последовательность символов
num_characters = 34  # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True)  # токенизируем на уровне символов
tokenizer.fit_on_texts([text])  # формируем токены на основе частотности в нашем тексте
print(tokenizer.word_index)

inp_chars = 10
data = tokenizer.texts_to_matrix(text)  # преобразуем исходный текст в массив OHE
n = data.shape[0] - inp_chars  # так как мы предсказываем по трем символам - четвертый

X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:]  # предсказание следующего символа

print(data.shape)

model = Sequential()
model.add(Input((inp_chars,
                 num_characters)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh'))  # рекуррентный слой на 500 нейронов
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=100)




{' ': 1, 'о': 2, 'а': 3, 'и': 4, 'е': 5, 'н': 6, 'т': 7, 'л': 8, 'с': 9, 'в': 10, 'м': 11, 'р': 12, 'к': 13, 'д': 14, 'у': 15, 'ы': 16, 'я': 17, 'п': 18, 'з': 19, 'ь': 20, 'б': 21, 'ч': 22, 'х': 23, 'г': 24, 'ш': 25, 'й': 26, 'ж': 27, 'ю': 28, 'ц': 29, 'э': 30, 'ф': 31, 'щ': 32, 'ъ': 33}
(8073, 34)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_7 (SimpleRNN)    (None, 128)               20864     
                                                                 
 dense_7 (Dense)             (None, 34)                4386      
                                                                 
Total params: 25250 (98.63 KB)
Trainable params: 25250 (98.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
252/252 [==============================] - 3s 6ms/step - loss: 2.9957 - accuracy: 0.1703
Epoch 2/100
2

In [ ]:
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))  # преобразуем символы в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters)

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении

        inp_str += d  # дописываем строку

    return inp_str


res = buildPhrase("программир")
print(res)

1/1 [==============================] - 0s 35ms/step
программирования для машинного обучения могут анализировать 
